# Candlesticks Patterns

Overlooking candlestick signals that major market participants rely on can mean missing valuable trading opportunities. The key is to view them as indicators of probabilities, not certainties. 

Technical formations can serve as effective position sizing and risk management triggers, particularly when aligned with fundamental catalysts or quantitative factors.

1. Pattern reliability varies significantly by trading session:
- Opening hour patterns often reflect overnight information processing
- End-of-day patterns frequently relate to institutional positioning

2. Large orders often create specific candlestick patterns due to VWAP execution algorithms.

3. Detecting market regime makes the pattern more reliable.

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import yfinance as yf
import pandas_ta as ta

Basic functions for the candlestick

In [2]:
class CandlestickBase():
    @staticmethod
    def calculate_body_length(df: pd.DataFrame) -> pd.Series:
        return (df["close"] - df["open"]).abs()
    
    @staticmethod
    def calculate_total_length(df: pd.DataFrame) -> pd.Series:
        return df['high'] - df['low']
    
    @staticmethod
    def calculate_upper_shadow(df: pd.DataFrame) -> pd.Series:
        return df["high"] - df[["open", "close"]].max(axis=1)

    @staticmethod
    def calculate_lower_shadow(df: pd.DataFrame) -> pd.Series:
        return df[["open", "close"]].min(axis=1) - df["low"]

    @staticmethod
    def calculate_trend(df: pd.DataFrame, window: int = 20) -> pd.Series:
        ma = df["close"].rolling(window=window).mean()
        trend = (df["close"] - ma) / ma
        return trend
    

Doji Candlestick

In [3]:
class Doji(CandlestickBase):
    def identify(self, df: pd.DataFrame) -> pd.Series:
        body_length = self.calculate_body_length(df)
        total_length = self.calculate_total_length(df)

        return (
            ((body_length / (total_length + 1e-7)) < 0.1) & # Small body
            ((body_length / df['close']) < 0.001)# Absolute body size
        )

Marubozu Candlestick

In [4]:
class Marubozu(CandlestickBase):
    def identify(self, df: pd.DataFrame) -> pd.Series:
        body_length = self.calculate_body_length(df)
        total_length = self.calculate_total_length(df)
        return ((body_length/total_length) > 0.98) 

Hammer Candlestick

In [5]:
class Hammer(CandlestickBase):
    def identify(self, df: pd.DataFrame) -> pd.Series:
        body_length = self.calculate_body_length(df)
        total_length = self.calculate_total_length(df)
        lower_shadow = self.calculate_lower_shadow(df)
        upper_shadow = self.calculate_upper_shadow(df)

        return (
            (lower_shadow > 2 * body_length) &# Lower shadow at least 2x body
            (upper_shadow < 0.3 * lower_shadow) &# Upper shadow less than 30% of lower
            ((body_length/total_length) < 0.3)# Not too large body
        )

Three White Soldiers Candlestick

In [6]:
class ThreeWhiteSoldiers(CandlestickBase):
    def identify(self, df: pd.DataFrame) -> pd.Series:
# Three consecutive up days
        up_days = df['close'] > df['open']
        consec_up = up_days & up_days.shift(1) & up_days.shift(2)

# Progressive higher opens and closes
        opens_within = (
            (df['open'] > df['open'].shift(1) * 0.99) &
            (df['open'] < df['close'].shift(1))
        )

# Volume consideration
        volume_increasing = (
            (df['volume'] > df['volume'].shift(1) * 0.95) &
            (df['volume'].shift(1) > df['volume'].shift(2) * 0.95)
        )

        return consec_up & opens_within & volume_increasing

Three Black Crows Candlestick

In [7]:
class ThreeBlackCrows(CandlestickBase):
	
    def identify(self, df: pd.DataFrame) -> pd.Series:
        trend = self.calculate_trend(df, window=10)

        down_days = df["close"] < df["open"]
        consec_down = down_days & down_days.shift(1) & down_days.shift(2)

        opens_within = (
            (df["open"] < df["open"].shift(1))
            & (df["open"] > df["close"].shift(1))
            & (df["open"].shift(1) < df["open"].shift(2))
            & (df["open"].shift(1) > df["close"].shift(2))
        )

        lower_closes = (df["close"] < df["close"].shift(1)) & (
            df["close"].shift(1) < df["close"].shift(2)
        )

        # Volume increasing for confirmation
        volume_increasing = (df["volume"] > df["volume"].shift(1)) & (
            df["volume"].shift(1) > df["volume"].shift(2)
        )

        return (
            consec_down
            & opens_within
            & lower_closes
            & volume_increasing
            & (trend.shift(3) > 0.02)
        )

Morning Star Candlestick

In [8]:
class MorningStar(CandlestickBase):
	
    def identify(self, df: pd.DataFrame) -> pd.Series:
        trend = self.calculate_trend(df)

# First day: long black body
        first_day = (df["close"].shift(2) < df["open"].shift(2))

# Second day: small body gapping down
        second_day = (
            self.calculate_body_length(df.shift(1)) < 0.5 * self.calculate_body_length(df.shift(2))
        ) & (df["high"].shift(1) < df["close"].shift(2))

# Third day: white body closing into first day's body
        third_day = (df["close"] > df["open"]) & (df["close"] > (df["open"].shift(2) + df["close"].shift(2)) / 2)

        return first_day & second_day & third_day & (trend.shift(3) < -0.02)

Evening Star Candlestick

In [9]:
class EveningStar(CandlestickBase):
	
    def identify(self, df: pd.DataFrame) -> pd.Series:

# First day: long black body
        first_day = (df["close"].shift(2) > df["open"].shift(2))

# Second day: small body gapping down
        second_day = (
            self.calculate_body_length(df.shift(1)) < 0.5 * self.calculate_body_length(df.shift(2))
        ) & (df["low"].shift(1) > df["close"].shift(2))

# Third day: white body closing into first day's body
        third_day = (df["close"] < df["open"]) & (df["close"] < (df["open"].shift(2) + df["close"].shift(2)) / 2)

        return first_day & second_day & third_day

Engulfing Candlestick

In [23]:
class BullishEngulfing(CandlestickBase):
	
    def identify(self, df: pd.DataFrame) -> pd.Series:
        return ((df["close"] > df["open"]) &  # Current candlestick is bullish
                (df["close"].shift(1) < df["open"].shift(1)) &  # Previous candlestick is bearish
                (df["open"] < df["close"].shift(1)) &  # Current open is less than previous close
                (df["close"] > df["open"].shift(1)))  # Current close is greater than previous open


class BearishEngulfing(CandlestickBase):
	
    def identify(self, df: pd.DataFrame) -> pd.Series:
        return ((df["close"] < df["open"]) &
                (df["close"].shift(1) > df["open"].shift(1)) &
                (df["open"] > df["close"].shift(1)) &
                (df["close"] < df["open"].shift(1)))

### Confidence in the pattern generated

In [11]:
class Confidence():
    
    def __init__(self, df: pd.DataFrame, window: int = 20, threshold: float = 0.7):
        self.df = df
        self.window = window    
        self.threshold = threshold

    def calculate_volume_ma(self) -> pd.Series:
        return self.df["volume"].rolling(window=self.window).mean()
    
    def calculate_volatility(self) -> pd.Series:
        return self.df["close"].rolling(window=self.window).std()
    
    def calculate_trend_strength(self):
        """
        Calculate the trend factor based on metrics like DI and ADX.
        
        :return: A binary trend factor (1 for strong trend, 0 for weak/no trend).
        """
        adx_threshold = 25  
        adx = ta.adx(self.df["high"], self.df["low"], self.df["close"], length=self.window) 
        trend_factor = (adx[f"ADX_{self.window}"] > adx_threshold).astype(int)
        return trend_factor
    
    def calculate_confidence(self) -> pd.Series:
        """
        Calculate the confidence score for candlestick pattern detection.
        The reliability of the pattern detected is determined by factors such as volume, volatility, and trend.
        This score ranges between 0 and 1, where higher values indicate more reliable signals.
        """
        # Volume Factor (30% weight)
        volume_factor = (self.df["volume"] > self.calculate_volume_ma()).astype(int)

        # Volatility Factor (30% weight)
        volatility = self.calculate_volatility()
        volatility_factor = (
            volatility > volatility.mean()
        ).astype(int)

        # Trend Factor (40% weight)
        trend_factor = self.calculate_trend_strength()

        # Combine factors into a confidence score
        confidence = (
            (volume_factor * 0.3)
            + (volatility_factor * 0.3)
            + (trend_factor * 0.4)
        )

        return confidence
    
    def validate_patterns(self) -> pd.Series:
        """
        Validate detected patterns based on their confidence scores.
        """
        return self.calculate_confidence() >= self.threshold

Get stock data

In [12]:
stock_ticker = 'AAPL'

stock_data = yf.download(stock_ticker, start='2018-01-01', end='2022-02-26')
stock_data.columns = stock_data.columns.str.lower()

[*********************100%%**********************]  1 of 1 completed


Plot the candlestick pattern

In [21]:
def plot_candlestick(stock_data : pd.DataFrame, candle : CandlestickBase) -> None:
    # Create the candlestick chart
    fig = go.Figure()

    # Add candlestick trace
    fig.add_trace(go.Candlestick(
        x=stock_data.index,
        open=stock_data["open"],
        high=stock_data["high"],
        low=stock_data["low"],
        close=stock_data["close"],
        name="OHLC"
    ))

    fig.add_trace(go.Scatter(
        x=stock_data.index,
        y=np.where(candle.identify(stock_data), stock_data["close"], np.nan) ,
        mode="markers",
        marker=dict(color="blue", size=4),
        name=f"{candle.__class__.__name__} Pattern"
    ))

    fig.add_trace(go.Scatter(
        x=stock_data.index,
        y=np.where(Confidence(stock_data).validate_patterns(), stock_data["close"].min(), np.nan) ,
        mode="markers",
        marker=dict(color="yellow", size=4),
        name="Reliable Detected Patterns"
    ))

    # Update layout
    fig.update_layout(
        title=f"Candlestick Chart with {candle.__class__.__name__} Patterns",
        xaxis_title="Date",
        yaxis_title="Price",
        xaxis_rangeslider_visible=False,  # Hide the default range slider
    )

    # Show the chart
    fig.show()

In [25]:
plot_candlestick(stock_data, BearishEngulfing())